Development notebook for debt fraction calculator. Use this to test code changes without re-running the scrape.

debt_fraction_calc.py has a few more comments and polish

In [ ]:
import pandas as pd
import numpy as np
import PySAM.Singleowner as singleowner
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd()))
from lcoe_calculator.process_all import ProcessAll
from lcoe_calculator.extractor import Extractor

from lcoe_calculator.tech_processors import FixedOffShoreWindProc, FloatingOffShoreWindProc, LandBasedWindProc, DistributedWindProc,\
    UtilityPvProc, CommPvProc, ResPvProc, UtilityPvPlusBatteryProc,\
    CspProc, GeothermalProc, HydropowerProc, PumpedStorageHydroProc,\
    NuclearProc, BiopowerProc

from lcoe_calculator.macrs import MACRS_6, MACRS_16, MACRS_21

In [ ]:
# Data master version on sharepoint - empty string if you haven't renamed the file
version_string = "_v1.46"

# Path to data master spreadsheet
data_master_filename = "path/to/2024-ATB-Data_Workbook.xlsx"

techs = [UtilityPvPlusBatteryProc
            ]


CRP_CHOICES = [20]
scraper = ProcessAll(data_master_filename, techs=techs)
scraper.process()

In [ ]:
df_itc, df_ptc = Extractor.get_tax_credits_sheet(data_master_filename)

In [ ]:
print(set(scraper.data.Parameter))

In [ ]:
fuel_filename = 'fuel_prices.csv'
df_fuel = pd.read_csv(fuel_filename).set_index("Price")
df_fuel.columns = df_fuel.columns.astype(int)
print(df_fuel)

In [ ]:
def calculate_debt_fraction(input_vals, debug=False):
    model = singleowner.default("GenericSystemSingleOwner")

    analysis_period = 20
    ac_capacity = 1000 # kW
    capacity_factor = input_vals["CF"]
    gen = [capacity_factor * ac_capacity] * 8760 # Distribute evenly throughout the year

    capex = input_vals["OCC"]
    con_fin_costs = input_vals["CFC"]
    initial_investment = capex * ac_capacity
    con_fin_total = con_fin_costs * ac_capacity
    o_and_m = input_vals["Fixed O&M"]
    v_o_and_m = input_vals["Variable O&M"]
    dscr = input_vals["DSCR"]

    ## Set these here so we can adjust below
    irr_target = input_vals["IRR"] 
    tax_federal = input_vals["Tax Rate (Federal and State)"] * 100
    tax_state = 0
    inflation = input_vals["Inflation Rate"] * 100

    degradation = 0.0 # ATB presents average capactity factors

    model.value("analysis_period", analysis_period)
    model.value("flip_target_year", analysis_period)
    model.value("gen", gen)
    model.value("system_pre_curtailment_kwac", gen)
    model.value("system_capacity", ac_capacity)
    model.value("cp_system_nameplate", ac_capacity / 1000)
    model.value("total_installed_cost", initial_investment)

    ## Single Owner should apply the O&M cost to each year, so no need to multiply by analysis period?
    model.value("om_capacity", [o_and_m] ) 
    model.value("om_production", [v_o_and_m])
    if 'Fuel' in input_vals:
        model.value("om_fuel_cost", [input_vals['Fuel']])
    if 'Heat Rate' in input_vals:
        model.value("system_heat_rate", input_vals['Heat Rate'])

    model.value("degradation", [degradation]) # Specify length 1 so degradation is applied each year. An array of 0.7 len(analysis_period) assumes degradation the first year, but not afterwards
    model.value("system_use_lifetime_output", 0) # Do degradation in the financial model

    model.value("debt_option", 1) # Use DSCR
    model.value("dscr", dscr)
    # model.value("debt_percent", 51.9)
    model.value("inflation_rate", inflation)
    model.value("term_int_rate", input_vals['Interest Rate Nominal'] * 100)
    model.value("term_tenor", 18)
    model.value("real_discount_rate", input_vals['Calculated Rate of Return on Equity Real'] * 100) ## "real equity rate" (also get this from data?)
    model.value("flip_target_percent", irr_target) ## "nominal equity rate"
    model.value("ppa_escalation", 0.0)

    model.value("federal_tax_rate", [tax_federal])
    model.value("state_tax_rate", [tax_state])

    # This group is included in fixed O&M
    model.value("insurance_rate", 0)
    model.value("property_tax_rate", 0)
    model.value("prop_tax_cost_assessed_percent", 0)

    model.value("reserves_interest", 0)
    model.value("salvage_percentage", 0)
    model.value("months_receivables_reserve", 0)
    model.value("months_working_reserve", 0)
    model.value("dscr_reserve_months", 0)
    model.value("equip1_reserve_cost", 0)
    model.value("equip2_reserve_cost", 0)
    model.value("equip3_reserve_cost", 0)
    model.value("cost_debt_closing", 0)
    model.value("cost_debt_fee", 0)
    model.value("loan_moratorium", 0)
    model.value("construction_financing_cost", con_fin_total)
    model.value("itc_fed_percent", [input_vals["ITC"] * 100])
    model.value('itc_fed_percent_maxvalue', [1e38])
    model.value("itc_sta_amount", [0])
    model.value("ptc_fed_amount", [input_vals["PTC"] / 1000]) # Convert $/MWh to $/kWh

    if input_vals["MACRS"] == MACRS_6:
        model.value("depr_alloc_macrs_5_percent", 100)
        model.value("depr_itc_fed_macrs_5", 1)
        model.value("depr_itc_sta_macrs_5", 1)
        model.value("depr_alloc_macrs_15_percent", 0)
        model.value("depr_alloc_sl_20_percent", 0)
    elif input_vals["MACRS"] == MACRS_16:
        model.value("depr_alloc_macrs_5_percent", 0)
        model.value("depr_alloc_macrs_15_percent", 100)
        model.value("depr_itc_fed_macrs_15", 1)
        model.value("depr_itc_sta_macrs_15", 1)
        model.value("depr_alloc_sl_20_percent", 0)
    elif input_vals["MACRS"] == MACRS_21:
        model.value("depr_alloc_macrs_5_percent", 0)
        model.value("depr_alloc_macrs_15_percent", 0)
        model.value("depr_alloc_sl_20_percent", 100)
        model.value("depr_itc_fed_sl_20", 1)
        model.value("depr_itc_fed_sl_20", 1)
    model.value("depr_alloc_custom_percent", 0)
    model.value("depr_alloc_sl_5_percent", 0)
    model.value("depr_alloc_sl_15_percent", 0)
    model.value("depr_alloc_sl_39_percent", 0)
    model.value("depr_bonus_fed", 0)
    model.value("depr_bonus_sta", 0)
    model.value("depr_bonus_fed_macrs_5", 0)
    model.value("depr_bonus_sta_macrs_5", 0)
    model.value("depr_bonus_fed_macrs_15", 0)
    model.value("depr_bonus_sta_macrs_15", 0)

    model.value("depr_fedbas_method", 0)
    model.value("depr_stabas_method", 0)

    model.value("ppa_soln_mode", 0)
    model.value("payment_option", 0)

    model.value('en_electricity_rates', 1 )

    model.execute()

    if debug:
        print("LCOE: " + str(model.Outputs.lcoe_real)) #Cents / kWh - multiply by 10 to get $ / MWh
        print("NPV: " + str(model.Outputs.project_return_aftertax_npv))
        print()
        print("IRR in target year: " + str(model.Outputs.flip_target_irr))
        print("IRR at end of project: " + str(model.Outputs.analysis_period_irr))
        print("O&M: " + str(model.Outputs.cf_om_capacity_expense))
        print("PPA price: " + str(model.Outputs.cf_ppa_price))
        print("Debt Principal: " + str(model.Outputs.cf_debt_payment_principal))
        print("Debt Interest: " + str(model.Outputs.cf_debt_payment_interest))
        print("Depreciation: " + str(model.Outputs.cf_feddepr_total))
        print("Production: " + str(model.Outputs.cf_energy_net))
        print("Tax " + str(model.Outputs.cf_fedtax))
        print("ITC " + str(model.Outputs.itc_total_fed))
        print("PTC " + str(model.Outputs.cf_ptc_fed))
        print("Debt fraction " + str(model.Outputs.debt_fraction))

    return model.Outputs.debt_fraction

In [ ]:
d = scraper.data
print(d.columns)
print(set(d.Parameter))

In [ ]:
fin_cases = ['Market','R&D']

d = scraper.data
debt_frac_dict = {}

years = range(2021, 2051)
print (years)
cols = ["Technology", "Case", *years]

for tech in techs:
    for fin_case in fin_cases:
        print("Fin case " , fin_case)
        debt_fracs = [tech.tech_name, fin_case]
        for year in years:
            input_vals = d[(d.DisplayName == tech.default_tech_detail) & (d.Case == fin_case) & (d.Scenario == 'Moderate') & (d.CRPYears == 20) & 
                ((d.Parameter == 'Fixed O&M') | (d.Parameter == 'Variable O&M') |(d.Parameter == 'OCC') | (d.Parameter == 'CFC') | (d.Parameter == 'CF')
                | (d.Parameter == 'Heat Rate') | (d.Parameter == 'Fuel'))]
            input_vals = input_vals.set_index('Parameter')[year].to_dict()

            gen_vals = d[(d.Technology == tech.tech_name) & (d.CRPYears == 20) & (d.Case == fin_case) & 
                ((d.Parameter == 'Inflation Rate') | (d.Parameter == 'Tax Rate (Federal and State)') | (d.Parameter == 'Calculated Rate of Return on Equity Real') | (d.Parameter == 'Interest Rate Nominal'))]
            gen_vals = gen_vals.set_index('Parameter')[year].to_dict()

            if tech.has_itc and tech.has_ptc and fin_case == 'Market':
                input_vals["PTC"] = df_ptc.loc[tech.sheet_name][year]
                input_vals["ITC"] = df_itc.loc[tech.sheet_name][year]
            else:
                input_vals["PTC"] = 0
                input_vals["ITC"] = 0
            input_vals["DSCR"] = tech.dscr
            input_vals["IRR"] = tech.irr_target
            if isinstance(tech.depreciation_schedule, list):
                input_vals["MACRS"] = tech.depreciation_schedule
            elif isinstance(tech.depreciation_schedule, dict):
                input_vals["MACRS"] = tech.depreciation_schedule[year]
            input_vals.update(gen_vals)
            debt_frac = calculate_debt_fraction(input_vals)

            debt_fracs.append(debt_frac / 100.0)
        
        debt_frac_dict[tech.tech_name + fin_case] = debt_fracs

df_df = pd.DataFrame.from_dict(debt_frac_dict, orient='index', columns=cols)
df_df.to_csv("2023_debt_fractions.csv")


In [ ]:
df_fuel.at['Natural Gas ($/MMBtu)', year]

In [ ]:
"""
if tech == NaturalGasProc:
    input_vals['Fuel'] = df_fuel.at['Natural Gas ($/MMBtu)', year]
if tech == CoalProc:
    input_vals['Fuel'] = df_fuel.at['Coal ($/MMBtu)', year]
"""

In [ ]:
print(input_vals)

print(debt_frac)